# 당근 데이터 전처리

## 0. data load & settings

In [6]:
import pandas as pd
import re

df = pd.read_csv('../../data/raw/daangn.csv')
raw = df
raw.head()

,id,title,detail,condition,uploaded_date,is_completed,price,location
0,fdffb5c6-480b-4891-b6d4-ff41b6fabd83,가격낮춤!(24년3월구매)스토케 익스플로리 엑스 디럭스 유모차,스토케 익스플로리 엑스 디럭스 유모차입니다. ☆보관장소가 협소해서 빠른거래위해 가격...,NaN,2024-11-24 15:15:35.209000,True,600000.0,천호동
1,98f6c307-e94f-4c78-9f6f-c1ab6045a2f3,가격내림제안주세요🩶스토케 비트 블랙멜란지 절충형 유모차 팔아요🩶+방풍커버,21년도에 아기 태어나면서 구매한 절충형 유모차 판매합니다 ! 백만원 넘게 주고 새...,NaN,2023-11-10 20:32:23.635000,True,200000.0,덕은동
2,d9036794-e558-46a6-a1e7-dbde3c900836,[가격내림] 스토케 스쿠트 유모차,스토케 스쿠트 유모차 구입일시: 2023년 7월 (재당근) 당근에서 구매한 스토케 ...,NaN,2024-05-29 21:07:26.426000,True,20000.0,등촌제2동
3,f02eaa6a-784b-4fa7-ab86-06edea7af02b,가격내림) 스토케 익스플로리 엑스 디럭스 유모차(거의새거),신세계 백화점 구매했는데 겨울에 구매해서 밖에 나가보지도 못하고 두고있다 둘째때 써...,NaN,2024-01-15 13:42:37.786000,True,650000.0,대저2동
4,18babe60-fa9e-4bad-a066-f8cd5c6eef56,(가격내림)스토케 크루시 디럭스 유모차,17년도 구매했습니다! 장바구니가 커서 사용하기 좋아요~ 유모차 전문업체 세탁 후 ...,NaN,2024-01-21 11:08:24.848000,True,45000.0,월성1동


## 1. condition
    1. "15,000원”과 같은 맞지 않는 정보가 있음 ⇒ 제거
    2. title + detail에서 상태값 생성 
        1. 네이버 카테고리에 맞춰서 추가 ⇒ 사용감 있음, 거의 새 것, 새 상품
        2. 구할 수 없는 경우에 따른 카테고리 추가  ⇒ DNN(너무 많으면 영향을 줄지도 고민)

## 2. location 
    1. 구로 묶어서 구별 가격 군집 분석 ⇒ 파일 기반으로 함수화하여 공통 처리
    2. 유아동의 경우 Nan으로 처리

## 3. title/detail 
    1. 이모지가 존재하므로 제거하고 사용
    2. detail에  html 태그만 들어간 경우 제거(소수)

In [ ]:
# html 제거용 정규식
html_pattern = re.compile(r'<.*?>')

# 이모지 제거용 정규식
emoji_pattern = re.compile("[" 
    u"\U0001F600-\U0001F64F"  # emoticons
    u"\U0001F300-\U0001F5FF"  # symbols & pictographs
    u"\U0001F680-\U0001F6FF"  # transport & map
    u"\U0001F1E0-\U0001F1FF"  # flags
    u"\u2700-\u27BF"          # dingbats
    u"\U0001F900-\U0001F9FF"  # supplemental symbols
    u"\U0001FA70-\U0001FAFF"  # extended-A
    u"\u2600-\u26FF"          # ★ 추가: Misc Symbols (☀, ☺ 등)
    u"\u2B50"                 # ★ 추가: ⭐
    u"\u2B55"                 # ★ 추가: ⭕
    u"\u203C"                 # ★ 추가: ‼
    u"\uFE0E-\uFE0F"          # ★ 추가: variation selectors (️ 잔여 제거)
    "]+", flags=re.UNICODE)

def clean_text(text):
    no_html = html_pattern.sub('', text)      # HTML 제거
    no_emoji = emoji_pattern.sub('', no_html) # 이모지 제거
    return no_emoji.strip()

df["title"] = df["title"].apply(clean_text)
df["detail"] = df["detail"].apply(clean_text)

# df.to_csv('check_clean.csv')